# RECOGIDA DE LOS DATOS<br>

En este apartado se tratarán los 3 datasets y almacenando la información en uno único:

- Importamos las _librerías_ necesarias:

In [1]:
import re
import time
import numpy as np

import random
import tldextract
import ipaddress
from urllib.parse import urlparse

from urllib.request import urlopen
import requests 
from bs4 import BeautifulSoup

from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

- __Directorios__ utilizados:

In [2]:
import os
PROJECT_ROOT_PATH = "."
DATASETS_PATH = PROJECT_ROOT_PATH + os.sep + "datasets"
FINAL_DATASETS_PATH = PROJECT_ROOT_PATH + os.sep + "final_datasets"

***

- Con la siguiente función leemos cada fichero CSV y devolvemos un _DataFrame_:<br>

In [3]:
import pandas as pd

def load_data(filename, separator, folder, path=DATASETS_PATH):
    file_path = os.path.join(path, folder + os.sep + filename)
    return pd.read_csv(file_path, sep=separator)

- Recogemos los datos de __tres fuentes__ diferentes, almacenando cada conjunto de datos en un _Dataframe_ diferente:<br>
__1.__ _df__urldata_, fichero __urldata.csv__ (fuente: https://www.kaggle.com/siddharthkumar25/malicious-and-benign-urls/kernels)<br>
__2.__ _df__malware_, fichero __Malware_dataset.csv__ (fuente: https://www.unb.ca/cic/datasets/url-2016.html)<br>
__3.__ _df__urlhaus__malicious_, fichero __urlhaus_dataset.csv__ (fuente: https://urlhaus.abuse.ch/api/#retrieve)<br>

In [4]:
df_urldata = load_data("urldata.csv",',', "malicious-and-benign-url-datasets")
df_malware = load_data("Malware_dataset.csv",',', "malicious-url-datasets")
df_urlhaus_malicious = load_data("urlhaus_dataset.csv",',', "malicious-url-datasets")

## __1.__ __urldata.csv__<br>
- Comprobamos el contenido del _Dataframe df__urldata_. Con la función _head_ en Pandas obtenemos las 5 primeras líneas del _DataFrame_ si no especificamos ningún otro valor:<br>

In [5]:
df_urldata.head()

,Unnamed: 0,url,label,result
0,0,https://www.google.com,benign,0
1,1,https://www.youtube.com,benign,0
2,2,https://www.facebook.com,benign,0
3,3,https://www.baidu.com,benign,0
4,4,https://www.wikipedia.org,benign,0


- Comprobamos la información del _DataFrame_:<br>

In [6]:
df_urldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417336 entries, 0 to 417335
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  417336 non-null  int64 
 1   url         417336 non-null  object
 2   label       417336 non-null  object
 3   result      417336 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 12.7+ MB


- Agrupamos los datos según los valores de la columna "label":<br>

In [7]:
df_urldata["label"].value_counts()

benign       345738
malicious     71598
Name: label, dtype: int64

Si agrupamos los datos según los valores de la columna __'label'__ (benign y malicious) comprobamos que existen 71598 URLs categorizadas como maliciosas y 345738 no maliciosas.<br>
Podemos ver que contiene __cuatro columnas__. De estas cuatro nos interesan solamente las columnas __'url'__ y __'label'__:<br>

- Eliminamos las columnas "Unnamed: 0" y "result":

In [8]:
#DataFrame con sólo columnas "url" y "label"
df_urldata = df_urldata.drop(['Unnamed: 0','result'], axis=1)
display(df_urldata.head())

#Agrupamos datos por columna "label"
display(df_urldata.groupby(['label']).count())

,url,label
0,https://www.google.com,benign
1,https://www.youtube.com,benign
2,https://www.facebook.com,benign
3,https://www.baidu.com,benign
4,https://www.wikipedia.org,benign


,url
label,
benign,345738
malicious,71598


## __2.__ Malware_dataset.csv<br>
- Comprobamos el contenido del _Dataframe df__malware_:<br>

In [9]:
df_malware.head()

,url
0,http://gzzax.livechatvalue.com/chat/chatClient...
1,http://gzzax.livechatvalue.com/chat/chatClient...
2,http://gzzax.livechatvalue.com/chat/chatClient...
3,http://gzzax.livechatvalue.com/chat/chatClient...
4,http://mtsx.com.cn/UploadFiles/2011-08/admin/%...


- Comprobamos la información del _DataFrame_:<br>

In [10]:
df_malware.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11566 entries, 0 to 11565
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     11566 non-null  object
dtypes: object(1)
memory usage: 90.5+ KB


Este _Dataframe_ solo contiene __una columna 'url'__.<br>
Este _Dataset_ consiste en una blacklist de URLs maliciosas, por lo que en todo el conjunto no encontraremos ninguna URL que no sea maliciosa. Añadimos una __nueva columna 'label'__ con el valor _'malicious'_ para cada una de las URLs:<br>

In [11]:
df_malware['label'] = "malicious"
df_malware.head()

,url,label
0,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
1,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
2,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
3,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
4,http://mtsx.com.cn/UploadFiles/2011-08/admin/%...,malicious


- Agrupamos los datos según los valores de la columna "label":<br>

In [12]:
df_malware["label"].value_counts()

malicious    11566
Name: label, dtype: int64

In [13]:
display(df_malware.head())
df_malware["label"] = 'malicious';display(df_malware.head())
display(df_malware.groupby(["label"]).count())

,url,label
0,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
1,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
2,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
3,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
4,http://mtsx.com.cn/UploadFiles/2011-08/admin/%...,malicious


,url,label
0,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
1,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
2,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
3,http://gzzax.livechatvalue.com/chat/chatClient...,malicious
4,http://mtsx.com.cn/UploadFiles/2011-08/admin/%...,malicious


,url
label,
malicious,11566


Si agrupamos los datos según los valores de la columna 'label' (solo _malicious_) comprobamos que existen __11566 URLs__ __maliciosas__.<br>

## __3.__ urlhaus_dataset.csv<br>
- Comprobamos el contenido del _Dataframe df__urlhaus__malicious_:<br>

In [14]:
df_urlhaus_malicious.head()

,id,dateadded,url,url_status,threat,tags,urlhaus_link,reporter
0,288338,2020-04-18 22:29:56,https://mitsui-jyuku.mixh.jp/uploads/3884d4fe8...,online,malware_download,exe,https://urlhaus.abuse.ch/url/344185/,p5yb34m
1,288337,2020-04-18 22:29:46,https://mitsui-jyuku.mixh.jp/uploads/3881q2w3e...,online,malware_download,exe,https://urlhaus.abuse.ch/url/344184/,p5yb34m
2,288336,2020-04-18 22:29:42,https://mitsui-jyuku.mixh.jp/uploads/387N8Mk0.exe,online,malware_download,exe,https://urlhaus.abuse.ch/url/344183/,p5yb34m
3,288335,2020-04-18 22:29:28,https://mitsui-jyuku.mixh.jp/uploads/3870nH6.exe,online,malware_download,exe,https://urlhaus.abuse.ch/url/344182/,p5yb34m
4,288334,2020-04-18 22:29:23,https://mitsui-jyuku.mixh.jp/uploads/386ODNO38...,online,malware_download,exe,https://urlhaus.abuse.ch/url/344181/,p5yb34m


- Comprobamos la información del _DataFrame_:<br>

In [15]:
df_urlhaus_malicious.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288339 entries, 0 to 288338
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            288339 non-null  int64 
 1   dateadded     288339 non-null  object
 2   url           288339 non-null  object
 3   url_status    288327 non-null  object
 4   threat        288295 non-null  object
 5   tags          288339 non-null  object
 6   urlhaus_link  288339 non-null  object
 7   reporter      288339 non-null  object
dtypes: int64(1), object(7)
memory usage: 17.6+ MB


Este conjunto de datos contiene todas las __URLs maliciosas__ recopiladas por la organización _URLhaus_.<br>
Este _Dataframe_ contiene __8 columnas__, de las cuales solo nos interesa la columna _'url'_.<br>

In [16]:
df_urlhaus_malicious = pd.DataFrame(df_urlhaus_malicious['url'])
df_urlhaus_malicious['label'] = "malicious"
df_urlhaus_malicious.head()

,url,label
0,https://mitsui-jyuku.mixh.jp/uploads/3884d4fe8...,malicious
1,https://mitsui-jyuku.mixh.jp/uploads/3881q2w3e...,malicious
2,https://mitsui-jyuku.mixh.jp/uploads/387N8Mk0.exe,malicious
3,https://mitsui-jyuku.mixh.jp/uploads/3870nH6.exe,malicious
4,https://mitsui-jyuku.mixh.jp/uploads/386ODNO38...,malicious


El _Dataframe_ inicial no contenía una columna _'label'_, por lo que la añadimos con el valor _malicious_ para todos los casos.<br>

- Agrupamos los datos según los valores de la columna "label":<br>

In [17]:
df_urlhaus_malicious['label'].value_counts()

malicious    288339
Name: label, dtype: int64

In [18]:
df_urlhaus_malicious = pd.DataFrame(df_urlhaus_malicious['url'])
df_urlhaus_malicious["label"] = 'malicious'
display(df_urlhaus_malicious.head())
display(df_urlhaus_malicious.groupby(["label"]).count())

,url,label
0,https://mitsui-jyuku.mixh.jp/uploads/3884d4fe8...,malicious
1,https://mitsui-jyuku.mixh.jp/uploads/3881q2w3e...,malicious
2,https://mitsui-jyuku.mixh.jp/uploads/387N8Mk0.exe,malicious
3,https://mitsui-jyuku.mixh.jp/uploads/3870nH6.exe,malicious
4,https://mitsui-jyuku.mixh.jp/uploads/386ODNO38...,malicious


,url
label,
malicious,288339


Si agrupamos los datos según los valores de la columna 'label' (solo _malicious_) comprobamos que existen __301181 URLs__ categorizadas como __maliciosas__.<br>

***
### Combinación de los datos en un sólo DataFrame<br>

- __Combinamos__ los datos de los __tres__ _Dataframes_ anteriores en uno solo llamado df:<br>

In [19]:
df = pd.concat([df_urldata, df_malware, df_urlhaus_malicious], axis=0)
df.groupby(['label']).count()

,url
label,
benign,345738
malicious,371503


Podemos ver que en el _Dataframe_ final existen __400899 URLs maliciosas__ y __345738 no maliciosas__. Como podemos ver existe una __mayor proporción__ de urls etiquetadas a __"benign"__ que a __"malicious"__.<br>

- Convertimos los valores de la etiqueta _label_ de _String_ a _Integer_:<br>
    1. __'malicious'__ = __1__
    2. __'benign'__ = __0__

In [20]:
df['label'] = df['label'].map({'malicious': 1, 'benign': 0})
df.groupby(['label']).count()

,url
label,
0,345738
1,371503


***
## Guardamos los datos<br>
- __Guardamos__ el contenido del _DataFrame_ final para realizar la extracción de características como siguiente paso:<br>

In [21]:
import pandas as pd

def save_data(dataframe, filename, separator, folder, path=FINAL_DATASETS_PATH):
    file_path = os.path.join(path, folder + os.sep + filename)
    dataframe.to_csv(file_path, sep=separator, index=False)

In [22]:
save_data(df, "1_urls_dataset.csv", ',', "1_recogida_datos")